In [7]:
# Importações essenciais para deep learning e processamento de dados
import os
import numpy as np
import pandas as pd
import cv2
from datetime import datetime, timedelta
import random
import time
import psutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from collections import Counter

# Configuração de reprodutibilidade
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.20.0
GPU disponível: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Configurações do experimento
IMG_SIZE = 224  # Tamanho da imagem (224x224 pixels)
BATCH_SIZE = 32  # Tamanho do lote para treinamento
EPOCHS = 100  # Número máximo de épocas
VALIDATION_SPLIT = 0.3  # 30% dos dados para validação

# Caminhos dos datasets
FER2013_PATH = r"src/data/FER2013"
RAF_DB_PATH = r"src/data/ RAF-DB"
DFEW_DB_PATH = r"src/data/DFEW"


# Mapeamento das 7 emoções básicas
EMOTION_LABELS = {
    'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 
    'neutral': 4, 'sadness': 5, 'surprise': 6
}

print("Configurações definidas:")
print(f"- Tamanho da imagem: {IMG_SIZE}x{IMG_SIZE}")
print(f"- Batch size: {BATCH_SIZE}")
print(f"- Épocas máximas: {EPOCHS}")
print(f"- Classes de emoção: {len(EMOTION_LABELS)}")

Configurações definidas:
- Tamanho da imagem: 96x96
- Batch size: 32
- Épocas máximas: 100
- Classes de emoção: 7


In [4]:
class TrainingMonitor:
    """
    Classe para monitorar desempenho computacional durante o treinamento.
    Essencial para experimentos científicos reproduzíveis.
    """
    
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.peak_memory_mb = 0
        self.initial_memory_mb = 0
        self.process = psutil.Process()
        
    def start_monitoring(self):
        """Inicia o monitoramento de tempo e memória"""
        self.start_time = time.time()
        self.initial_memory_mb = self._get_memory_usage()
        self.peak_memory_mb = self.initial_memory_mb
        print(f"Iniciando treinamento ResNet50...")
        print(f"Horário de início: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Memória inicial: {self.initial_memory_mb:.2f} MB")
        print("-" * 50)
        
    def _get_memory_usage(self):
        """Retorna uso atual de memória em MB"""
        return self.process.memory_info().rss / 1024 / 1024
        
    def update_peak_memory(self):
        """Atualiza o pico de memória se necessário"""
        current_memory = self._get_memory_usage()
        if current_memory > self.peak_memory_mb:
            self.peak_memory_mb = current_memory

# Instancia o monitor
monitor = TrainingMonitor()

In [ ]:
def load_preprocessed_data_resnet50_from_images():
    """
    Carrega dados pré-processados de imagens JPG com normalização específica para ResNet50.
    ResNet50 usa normalização [0, 1] padrão.
    
    Estrutura esperada:
    data/processed/raf_db_temp_gray_aligned/
    ├── Raiva/
    ├── Nojo/
    ├── Medo/
    ├── Felicidade/
    ├── Neutro/
    ├── Tristeza/
    └── Surpresa/
    """
    import cv2
    import os
    import numpy as np
    from collections import Counter
    from sklearn.model_selection import train_test_split
    
    print("Carregando dados pré-processados JPG para ResNet50...")
    
    # Configurações
    IMG_SIZE = 224  # Tamanho padrão para ResNet50
    BASE_PATH = r".\data\processed\raf_db_temp_gray_aligned"  # Ajuste para seu caminho
    
    # Mapeamento das emoções em português
    EMOTION_LABELS = {
        'Raiva': 0, 'Nojo': 1, 'Medo': 2, 'Felicidade': 3, 
        'Neutro': 4, 'Tristeza': 5, 'Surpresa': 6
    }
    
    def load_images_from_directory(directory_path, set_name):
        """Carrega imagens de um diretório usando os.path.join"""
        images = []
        labels = []
        
        print(f"Carregando {set_name} de: {directory_path}")
        
        # Verifica se o diretório existe
        if not os.path.exists(directory_path):
            print(f"❌ Diretório não encontrado: {directory_path}")
            return np.array([]), np.array([])
        
        # Lista subdiretórios (emoções)
        subdirs = [d for d in os.listdir(directory_path) 
                  if os.path.isdir(os.path.join(directory_path, d))]
        
        print(f"📁 Subdiretórios encontrados: {subdirs}")
        
        for emotion, label in EMOTION_LABELS.items():
            # Usar os.path.join ao invés de /
            emotion_path = os.path.join(directory_path, emotion)
            
            if not os.path.exists(emotion_path):
                print(f"⚠️  Pasta '{emotion}' não encontrada em {directory_path}")
                print(f"    Tentando variações de nome...")
                
                # Tenta variações do nome da emoção
                emotion_variations = [
                    emotion.lower(),
                    emotion.upper(), 
                    emotion.capitalize(),
                    emotion.replace('ç', 'c'),  # Felicidade -> Felicidade
                    emotion.replace('ã', 'a')   # Raiva -> Raiva
                ]
                
                found = False
                for variation in emotion_variations:
                    test_path = os.path.join(directory_path, variation)
                    if os.path.exists(test_path):
                        emotion_path = test_path
                        print(f"    ✅ Encontrado: {variation}")
                        found = True
                        break
                
                if not found:
                    print(f"    ❌ Nenhuma variação encontrada para '{emotion}'")
                    continue
            
            # Carrega imagens da pasta da emoção
            count = 0
            image_files = []
            
            # Busca diferentes extensões
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp']:
                import glob
                pattern = os.path.join(emotion_path, ext)
                image_files.extend(glob.glob(pattern))
            
            print(f"  📸 {emotion}: {len(image_files)} arquivos encontrados")
            
            for img_file in image_files:
                try:
                    # Carrega imagem
                    img = cv2.imread(img_file)
                    if img is None:
                        print(f"    ⚠️ Não foi possível carregar: {os.path.basename(img_file)}")
                        continue
                    
                    # Converte BGR para RGB
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    
                    # Redimensiona se necessário
                    if img.shape[:2] != (IMG_SIZE, IMG_SIZE):
                        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
                    
                    # Garante que seja RGB (3 canais)
                    if len(img.shape) == 2:
                        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                    elif img.shape[2] == 1:
                        img = np.repeat(img, 3, axis=2)
                    elif img.shape[2] == 4:  # RGBA
                        img = img[:, :, :3]  # Remove canal alpha
                    
                    images.append(img)
                    labels.append(label)
                    count += 1
                    
                except Exception as e:
                    print(f"    ❌ Erro ao carregar {os.path.basename(img_file)}: {e}")
                    continue
            
            print(f"  ✅ {emotion}: {count} imagens carregadas com sucesso")
        
        return np.array(images), np.array(labels)
    
    def detect_data_structure(base_path):
        """Detecta a estrutura dos dados automaticamente"""
        print(f"🔍 Analisando estrutura de: {base_path}")
        
        if not os.path.exists(base_path):
            print(f"❌ Caminho base não existe: {base_path}")
            return None
            
        # Lista conteúdo do diretório
        contents = os.listdir(base_path)
        dirs = [d for d in contents if os.path.isdir(os.path.join(base_path, d))]
        files = [f for f in contents if os.path.isfile(os.path.join(base_path, f))]
        
        print(f"📁 Diretórios: {dirs}")
        print(f"📄 Arquivos: {len(files)} encontrados")
        
        # Verifica se tem estrutura train/test
        if 'train' in dirs and 'test' in dirs:
            print("✅ Estrutura detectada: train/test/emotion/")
            return 'train_test'
        
        # Verifica se as pastas são emoções diretamente
        emotion_names = set(EMOTION_LABELS.keys())
        found_emotions = set(dirs) & emotion_names
        
        if found_emotions:
            print(f"✅ Estrutura detectada: emotion/ direta - Emoções: {found_emotions}")
            return 'emotion_direct'
        
        # Verifica variações de nomes
        emotion_variations = []
        for emotion in EMOTION_LABELS.keys():
            variations = [emotion.lower(), emotion.upper(), emotion.capitalize()]
            emotion_variations.extend(variations)
        
        found_variations = set(dirs) & set(emotion_variations)
        if found_variations:
            print(f"✅ Estrutura detectada: emotion/ com variações - Encontradas: {found_variations}")
            return 'emotion_direct'
        
        print("⚠️ Estrutura não reconhecida automaticamente")
        return 'unknown'
    
    try:
        # Detecta estrutura automaticamente
        structure = detect_data_structure(BASE_PATH)
        
        if structure == 'train_test':
            # Estrutura: base/train/emotion/ e base/test/emotion/
            train_path = os.path.join(BASE_PATH, "train")
            test_path = os.path.join(BASE_PATH, "test")
            
            X_train, y_train = load_images_from_directory(train_path, "TREINO")
            X_test, y_test = load_images_from_directory(test_path, "TESTE")
            
        elif structure == 'emotion_direct':
            # Estrutura: base/emotion/ - precisa criar train/test split
            print("📊 Carregando todas as imagens e criando divisão train/test...")
            
            all_images, all_labels = load_images_from_directory(BASE_PATH, "TODAS AS IMAGENS")
            
            if len(all_images) == 0:
                print("❌ Nenhuma imagem carregada!")
                return None, None, None, None
            
            # Cria divisão train/test
            X_train, X_test, y_train, y_test = train_test_split(
                all_images, all_labels,
                test_size=0.2,
                stratify=all_labels,
                random_state=42
            )
            
            print("✅ Divisão train/test criada automaticamente (80/20)")
            
        else:
            print("❌ Estrutura de dados não suportada!")
            print("💡 Estruturas esperadas:")
            print("   1. base/train/Raiva/*.jpg, base/train/Nojo/*.jpg, etc.")
            print("   2. base/Raiva/*.jpg, base/Nojo/*.jpg, etc.")
            return None, None, None, None
        
        if len(X_train) == 0 or len(X_test) == 0:
            print("❌ Nenhuma imagem carregada. Verifique os caminhos e nomes das pastas!")
            return None, None, None, None
        
        print(f"\n📊 Dados carregados com sucesso:")
        print(f"- X_train: {X_train.shape}")
        print(f"- y_train: {y_train.shape}")
        print(f"- X_test: {X_test.shape}")
        print(f"- y_test: {y_test.shape}")
        
        # NORMALIZAÇÃO ESPECÍFICA PARA RESNET50: [0, 255] -> [0, 1]
        print("🔄 Aplicando normalização ResNet50...")
        
        X_train = X_train.astype(np.float32)
        X_test = X_test.astype(np.float32)
        
        # ResNet50 normalização padrão: [0, 255] -> [0, 1]
        X_train = X_train / 255.0
        X_test = X_test / 255.0
        
        print("✅ Normalização ResNet50 aplicada: [0,255] -> [0,1]")
        
        # Verifica resultado final
        print(f"\n🔍 Verificação final:")
        print(f"- X_train range: [{X_train.min():.3f}, {X_train.max():.3f}]")
        print(f"- X_test range: [{X_test.min():.3f}, {X_test.max():.3f}]")
        print(f"- Formato das imagens: {X_train.shape[1:]} (deve ser {IMG_SIZE}x{IMG_SIZE}x3)")
        
        # Verifica distribuição de classes
        train_distribution = dict(Counter(y_train))
        test_distribution = dict(Counter(y_test))
        
        print(f"\n📈 Distribuição de classes:")
        emotion_names = list(EMOTION_LABELS.keys())
        print("- Treino:")
        for label, count in train_distribution.items():
            emotion_name = emotion_names[label]
            print(f"  {emotion_name}: {count} imagens")
        
        print("- Teste:")
        for label, count in test_distribution.items():
            emotion_name = emotion_names[label]
            print(f"  {emotion_name}: {count} imagens")
        
        return X_train, y_train, X_test, y_test
        
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        print(f"📍 Erro detalhado: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        
        print("\n💡 Soluções possíveis:")
        print("1. Verifique se o caminho está correto")
        print("2. Verifique se as pastas de emoções existem")
        print("3. Verifique se há imagens nas pastas")
        print("4. Verifique permissões de acesso")
        
        return None, None, None, None

# Executa carregamento para ResNet50
X_train, y_train, X_test, y_test = load_preprocessed_data_resnet50_from_images()
monitor.update_peak_memory()

if X_train is not None:
    print(f"\n🎯 ResNet50: Dados prontos para treinamento!")
    print(f"📏 Formato final: {X_train.shape}")
    print(f"🎨 Range de valores: [{X_train.min():.3f}, {X_train.max():.3f}]")
    print(f"✅ Pronto para ResNet50!")
else:
    print("❌ Falha no carregamento dos dados")

In [ ]:
def create_experiment_structure():
    """
    Cria estrutura de diretórios para salvar modelos e métricas.
    """
    # Cria timestamp único para o experimento
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_id = f"resnet50_emotion_{timestamp}"
    
    # Cria diretórios
    os.makedirs("models", exist_ok=True)
    os.makedirs("metrics", exist_ok=True)
    os.makedirs("plots", exist_ok=True)
    
    return experiment_id

def save_model_if_good_performance(model, accuracy, f1_score, experiment_id, threshold=0.85):
    """
    Salva modelo apenas se a performance for boa.
    
    Args:
        model: Modelo treinado
        accuracy: Acurácia do modelo
        f1_score: F1-score macro do modelo
        experiment_id: ID único do experimento
        threshold: Limite mínimo para salvar (default: 85%)
    """
    if accuracy >= threshold or f1_score >= threshold:
        model_path = f"models/resnet50_emotion_{experiment_id}.pkl"
        
        # Salva apenas os pesos para economia de espaço
        model.save_weights(f"models/weights_resnet50_{experiment_id}.h5")
        
        # Salva configuração do modelo
        model_config = {
            'architecture': 'ResNet50',
            'img_size': IMG_SIZE,
            'num_classes': 7,
            'experiment_id': experiment_id,
            'accuracy': accuracy,
            'f1_score': f1_score,
            'timestamp': datetime.now().isoformat()
        }
        
        with open(f"models/config_resnet50_{experiment_id}.pkl", 'wb') as f:
            pickle.dump(model_config, f)
        
        print(f"Modelo salvo! Performance: Acc={accuracy:.4f}, F1={f1_score:.4f}")
        return True
    else:
        print(f"Performance insuficiente para salvar. Acc={accuracy:.4f}, F1={f1_score:.4f} < {threshold}")
        return False

def save_metrics_to_csv(metrics_dict, experiment_id):
    """
    Salva métricas de performance em arquivo CSV.
    
    Args:
        metrics_dict: Dicionário com todas as métricas
        experiment_id: ID único do experimento
    """
    # Converte métricas para DataFrame
    metrics_df = pd.DataFrame([metrics_dict])
    
    # Arquivo CSV principal
    csv_path = "metrics/performance_metrics.csv"
    
    # Append ao CSV se já existir, senão cria novo
    if os.path.exists(csv_path):
        metrics_df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        metrics_df.to_csv(csv_path, index=False)
    
    # Salva também arquivo individual do experimento
    individual_csv = f"metrics/metrics_{experiment_id}.csv"
    metrics_df.to_csv(individual_csv, index=False)
    
    print(f"Métricas salvas em: {csv_path} e {individual_csv}")

# Inicializa estrutura do experimento
experiment_id = create_experiment_structure()
print(f"Experimento iniciado: {experiment_id}")

In [ ]:
def create_resnet50_model():
    """
    Cria modelo ResNet50 com transfer learning para classificação de emoções.
    
    Arquitetura:
    - ResNet50 pré-treinado (ImageNet) como feature extractor
    - Global Average Pooling para redução dimensional
    - Camadas densas para classificação final
    - Dropout para regularização
    
    Returns:
        keras.Model: Modelo compilado pronto para treinamento
    """
    # Carrega ResNet50 pré-treinado no ImageNet
    base_model = ResNet50(
        weights='imagenet',          # Pesos pré-treinados
        include_top=False,           # Remove camadas de classificação originais
        input_shape=(IMG_SIZE, IMG_SIZE, 3)  # Define formato de entrada
    )
    
    # Congela camadas base para transfer learning
    base_model.trainable = False
    
    # Adiciona camadas de classificação customizadas
    x = base_model.output
    x = GlobalAveragePooling2D()(x)           # Reduz dimensionalidade espacial
    x = Dense(512, activation='relu')(x)       # Primeira camada densa
    x = Dropout(0.5)(x)                       # Regularização forte
    x = Dense(256, activation='relu')(x)       # Segunda camada densa
    x = Dropout(0.3)(x)                       # Regularização moderada
    predictions = Dense(7, activation='softmax')(x)  # Classificação final (7 emoções)
    
    # Cria modelo final
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compilação otimizada para classificação multiclasse
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Verifica se os dados foram carregados corretamente antes de criar o modelo
if X_train is not None:
    print("Criando modelo ResNet50 otimizado...")
    model, base_model = create_resnet50_model()
    
    print(f"Modelo criado com sucesso:")
    print(f"- Total de parâmetros: {model.count_params():,}")
    print(f"- Parâmetros treináveis: {sum([tf.keras.backend.count_params(p) for p in model.trainable_weights]):,}")
    print(f"- Camadas congeladas: {len([l for l in base_model.layers if not l.trainable])}")
    
    monitor.update_peak_memory()
else:
    print("Erro: Dados não carregados. Verifique a célula anterior.")

In [ ]:
class MemoryCallback(tf.keras.callbacks.Callback):
    """
    Callback customizado para monitoramento de memória durante treinamento.
    Essencial para experimentos com recursos limitados.
    """
    
    def __init__(self, monitor):
        super().__init__()
        self.monitor = monitor
        
    def on_epoch_end(self, epoch, logs=None):
        self.monitor.update_peak_memory()
        if epoch % 5 == 0:  # Log a cada 5 épocas
            current_memory = self.monitor._get_memory_usage()
            print(f"Época {epoch+1} - Memória atual: {current_memory:.2f} MB")

def setup_training_callbacks(monitor):
    """
    Configura callbacks para treinamento otimizado.
    
    Returns:
        list: Lista de callbacks configurados
    """
    # Early Stopping: Para evitar overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss',           # Métrica para monitorar
        patience=15,                  # Épocas sem melhoria antes de parar
        restore_best_weights=True,    # Restaura melhores pesos
        verbose=1,                    # Mostra quando para
        mode='min'                    # Minimizar loss
    )
    
    # Reduce Learning Rate: Ajuste adaptativo da taxa de aprendizado
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',           # Métrica para monitorar
        factor=0.2,                   # Fator de redução (lr = lr * factor)
        patience=10,                  # Épocas sem melhoria antes de reduzir
        min_lr=1e-7,                  # Taxa mínima de aprendizado
        verbose=1,                    # Mostra quando reduz
        mode='min'
    )
    
    # Memory monitoring callback
    memory_callback = MemoryCallback(monitor)
    
    return [early_stopping, reduce_lr, memory_callback]

# Preparação dos dados para treinamento
print("Preparando dados para treinamento...")

# Divisão treino/validação estratificada
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, 
    test_size=VALIDATION_SPLIT,   # 30% para validação
    stratify=y_train,             # Mantém proporção por classe
    random_state=42               # Reprodutibilidade
)

# Converte labels para formato categorical (one-hot encoding)
y_train_cat = to_categorical(y_train_split, 7)
y_val_cat = to_categorical(y_val, 7)
y_test_cat = to_categorical(y_test, 7)

print(f"Dados de treino: {X_train_split.shape}")
print(f"Dados de validação: {X_val.shape}")
print(f"Dados de teste: {X_test.shape}")
print(f"Distribuição de classes - Treino: {dict(Counter(y_train_split))}")
print(f"Distribuição de classes - Validação: {dict(Counter(y_val))}")

# Configura callbacks
callbacks = setup_training_callbacks(monitor)
print("Callbacks configurados para treinamento otimizado")

In [ ]:
def train_resnet50_model(model, X_train, y_train, X_val, y_val, monitor, callbacks):
    """
    Executa treinamento completo do ResNet50 com monitoramento detalhado.
    
    Returns:
        tuple: (history, training_metrics)
    """
    print("Iniciando treinamento ResNet50...")
    monitor.start_monitoring()
    
    # Inicia cronômetro específico do treinamento
    training_start_time = time.time()
    
    # Executa treinamento
    history = model.fit(
        X_train, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )
    
    # Calcula tempo de treinamento
    training_end_time = time.time()
    training_duration = training_end_time - training_start_time
    
    # Métricas do treinamento
    training_metrics = {
        'training_time_seconds': training_duration,
        'training_time_formatted': str(timedelta(seconds=int(training_duration))),
        'epochs_completed': len(history.history['accuracy']),
        'best_train_accuracy': max(history.history['accuracy']),
        'best_val_accuracy': max(history.history['val_accuracy']),
        'final_train_loss': history.history['loss'][-1],
        'final_val_loss': history.history['val_loss'][-1]
    }
    
    print(f"Treinamento concluído em: {training_metrics['training_time_formatted']}")
    print(f"Épocas executadas: {training_metrics['epochs_completed']}")
    print(f"Melhor acurácia validação: {training_metrics['best_val_accuracy']:.4f}")
    
    return history, training_metrics

# Preparação dos dados se carregamento foi bem-sucedido
if X_train is not None and y_train is not None:
    
    # Divisão estratificada treino/validação
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train, y_train,
        test_size=VALIDATION_SPLIT,
        stratify=y_train,
        random_state=42
    )
    
    # Conversão para categorical (one-hot encoding)
    y_train_cat = to_categorical(y_train_split, 7)
    y_val_cat = to_categorical(y_val, 7)
    y_test_cat = to_categorical(y_test, 7)
    
    print(f"Preparação concluída:")
    print(f"- Treino: {X_train_split.shape} | Labels: {y_train_cat.shape}")
    print(f"- Validação: {X_val.shape} | Labels: {y_val_cat.shape}")
    print(f"- Teste: {X_test.shape} | Labels: {y_test_cat.shape}")
    
    # Configura callbacks
    callbacks = setup_training_callbacks(monitor)
    
    # Executa treinamento
    history, training_metrics = train_resnet50_model(
        model, X_train_split, y_train_cat, X_val, y_val_cat, monitor, callbacks
    )
    
    print("Treinamento finalizado com sucesso!")
    
else:
    print("Erro: Dados não disponíveis para treinamento")

In [ ]:
def comprehensive_model_evaluation(model, X_test, y_test_cat, y_test_original):
    """
    Avaliação completa do modelo com todas as métricas necessárias.
    
    Returns:
        dict: Dicionário completo com todas as métricas
    """
    print("Iniciando avaliação completa do modelo...")
    
    # Mede tempo de inferência
    inference_start = time.time()
    y_pred_prob = model.predict(X_test, batch_size=BATCH_SIZE, verbose=1)
    inference_end = time.time()
    
    # Calcula métricas de tempo
    total_inference_time = inference_end - inference_start
    inference_per_sample = total_inference_time / len(X_test)
    samples_per_second = len(X_test) / total_inference_time
    
    # Conversões para cálculos de métricas
    y_pred_classes = np.argmax(y_pred_prob, axis=1)
    y_true_classes = y_test_original
    
    # Métricas de classificação
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true_classes, y_pred_classes, average='macro', zero_division=0
    )
    
    # Métricas por classe
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_true_classes, y_pred_classes, average='micro', zero_division=0
    )
    
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true_classes, y_pred_classes, average='weighted', zero_division=0
    )
    
    # Matriz de confusão
    conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
    
    # Métricas de memória atual
    current_memory = monitor._get_memory_usage()
    
    # Relatório detalhado por classe
    emotion_names = list(EMOTION_LABELS.keys())
    class_report = classification_report(
        y_true_classes, y_pred_classes, 
        target_names=emotion_names, 
        output_dict=True
    )
    
    # Compilação completa das métricas
    comprehensive_metrics = {
        # Identificação do experimento
        'experiment_id': experiment_id,
        'model_architecture': 'ResNet50',
        'timestamp': datetime.now().isoformat(),
        
        # Configurações do modelo
        'img_size': IMG_SIZE,
        'batch_size': BATCH_SIZE,
        'epochs_trained': len(history.history['accuracy']),
        
        # Métricas de performance principal
        'test_accuracy': accuracy,
        'f1_score_macro': f1,
        'f1_score_micro': f1_micro,
        'f1_score_weighted': f1_weighted,
        'precision_macro': precision,
        'recall_macro': recall,
        
        # Métricas de tempo
        'total_inference_time_seconds': total_inference_time,
        'inference_per_sample_ms': inference_per_sample * 1000,
        'samples_per_second': samples_per_second,
        'training_time_seconds': training_metrics['training_time_seconds'],
        
        # Métricas de memória
        'peak_memory_mb': monitor.peak_memory_mb,
        'current_memory_mb': current_memory,
        'memory_efficiency': monitor.initial_memory_mb / monitor.peak_memory_mb,
        
        # Métricas por classe (emotion-wise)
        'anger_f1': class_report['anger']['f1-score'],
        'disgust_f1': class_report['disgust']['f1-score'],
        'fear_f1': class_report['fear']['f1-score'],
        'happy_f1': class_report['happy']['f1-score'],
        'neutral_f1': class_report['neutral']['f1-score'],
        'sadness_f1': class_report['sadness']['f1-score'],
        'surprise_f1': class_report['surprise']['f1-score'],
        
        # Dados do dataset
        'train_samples': len(X_train_split),
        'val_samples': len(X_val),
        'test_samples': len(X_test),
        'total_parameters': model.count_params(),
        'trainable_parameters': sum([tf.keras.backend.count_params(p) for p in model.trainable_weights])
    }
    
    return comprehensive_metrics, conf_matrix, class_report

# Executa avaliação completa se treinamento foi bem-sucedido
if 'history' in locals() and history is not None:
    
    print("Executando avaliação completa...")
    
    # Avaliação detalhada
    metrics, confusion_matrix_result, detailed_report = comprehensive_model_evaluation(
        model, X_test, y_test_cat, y_test
    )
    
    # Salva métricas em CSV
    save_metrics_to_csv(metrics, experiment_id)
    
    # Tenta salvar modelo se performance for boa
    model_saved = save_model_if_good_performance(
        model, 
        metrics['test_accuracy'], 
        metrics['f1_score_macro'], 
        experiment_id,
        threshold=0.80  # Ajuste conforme necessário
    )
    
    # Finaliza monitoramento
    monitor_final_stats = monitor.end_monitoring()
    
    print(f"\nRESUMO DO EXPERIMENTO {experiment_id}:")
    print(f"Acurácia: {metrics['test_accuracy']:.4f}")
    print(f"F1-Score Macro: {metrics['f1_score_macro']:.4f}")
    print(f"Tempo de treinamento: {training_metrics['training_time_formatted']}")
    print(f"Inferência por amostra: {metrics['inference_per_sample_ms']:.2f} ms")
    print(f"Modelo salvo: {'Sim' if model_saved else 'Não'}")
    
else:
    print("Erro: Treinamento não foi executado corretamente")

In [ ]:
def create_comprehensive_visualizations(history, confusion_matrix_result, metrics, detailed_report):
    """
    Cria visualizações completas dos resultados para análise científica.
    """
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Histórico de treinamento
    ax1 = plt.subplot(2, 3, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.title('Model Accuracy - ResNet50', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    ax2 = plt.subplot(2, 3, 2)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title('Model Loss - ResNet50', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 2. Matriz de confusão
    ax3 = plt.subplot(2, 3, 3)
    emotion_names = list(EMOTION_LABELS.keys())
    sns.heatmap(confusion_matrix_result, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_names, yticklabels=emotion_names, ax=ax3)
    plt.title('Confusion Matrix - ResNet50', fontsize=14, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # 3. F1-Score por classe
    ax4 = plt.subplot(2, 3, 4)
    f1_scores = [detailed_report[emotion]['f1-score'] for emotion in emotion_names]
    bars = plt.bar(emotion_names, f1_scores, color='skyblue', edgecolor='navy', alpha=0.7)
    plt.title('F1-Score por Emoção', fontsize=14, fontweight='bold')
    plt.ylabel('F1-Score')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    
    # Adiciona valores nas barras
    for bar, score in zip(bars, f1_scores):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{score:.3f}', ha='center', va='bottom')
    
    # 4. Métricas de performance
    ax5 = plt.subplot(2, 3, 5)
    performance_metrics = ['Accuracy', 'F1-Macro', 'Precision', 'Recall']
    performance_values = [
        metrics['test_accuracy'],
        metrics['f1_score_macro'],
        metrics['precision_macro'],
        metrics['recall_macro']
    ]
    bars = plt.bar(performance_metrics, performance_values, 
                  color=['green', 'blue', 'orange', 'red'], alpha=0.7)
    plt.title('Métricas Gerais de Performance', fontsize=14, fontweight='bold')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    
    for bar, value in zip(bars, performance_values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom')
    
    # 5. Análise de tempo e memória
    ax6 = plt.subplot(2, 3, 6)
    resource_data = {
        'Tempo Treino (min)': metrics['training_time_seconds'] / 60,
        'Inferência/amostra (ms)': metrics['inference_per_sample_ms'],
        'Pico Memória (GB)': metrics['peak_memory_mb'] / 1024,
        'Amostras/seg': metrics['samples_per_second']
    }
    
    # Normaliza valores para visualização
    normalized_values = []
    labels = []
    for key, value in resource_data.items():
        if 'Tempo' in key:
            normalized_values.append(value / max(1, value) if value > 0 else 0)
        elif 'Memória' in key:
            normalized_values.append(min(value, 1))
        else:
            normalized_values.append(min(value / 100, 1))  # Normaliza outras métricas
        labels.append(f'{key}\n{value:.2f}')
    
    plt.bar(range(len(labels)), normalized_values, color='purple', alpha=0.7)
    plt.title('Recursos Computacionais (Normalizado)', fontsize=14, fontweight='bold')
    plt.xticks(range(len(labels)), [label.split('\n')[0] for label in labels], rotation=45)
    plt.ylabel('Valor Normalizado')
    
    plt.tight_layout()
    plt.savefig(f'plots/comprehensive_analysis_{experiment_id}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
    # Relatório textual final
    print(f"\n{'='*80}")
    print(f"RELATÓRIO CIENTÍFICO FINAL - EXPERIMENTO {experiment_id}")
    print(f"{'='*80}")
    print(f"ARQUITETURA: ResNet50 Transfer Learning")
    print(f"DATASET: Emoções balanceadas (7 classes)")
    print(f"CONFIGURAÇÃO: {IMG_SIZE}x{IMG_SIZE}, batch_size={BATCH_SIZE}")
    print(f"\nPERFORMANCE PRINCIPAL:")
    print(f"  • Acurácia de Teste: {metrics['test_accuracy']:.4f} ({metrics['test_accuracy']*100:.2f}%)")
    print(f"  • F1-Score Macro: {metrics['f1_score_macro']:.4f}")
    print(f"  • Precisão Macro: {metrics['precision_macro']:.4f}")
    print(f"  • Recall Macro: {metrics['recall_macro']:.4f}")
    print(f"\nEFICIÊNCIA COMPUTACIONAL:")
    print(f"  • Tempo de Treinamento: {training_metrics['training_time_formatted']}")
    print(f"  • Épocas Executadas: {metrics['epochs_trained']}")
    print(f"  • Inferência por Amostra: {metrics['inference_per_sample_ms']:.2f} ms")
    print(f"  • Throughput: {metrics['samples_per_second']:.1f} amostras/segundo")
    print(f"  • Pico de Memória: {metrics['peak_memory_mb']:.1f} MB")
    print(f"\nPERFORMANCE POR EMOÇÃO:")
    for emotion in emotion_names:
        f1_score = detailed_report[emotion]['f1-score']
        print(f"  • {emotion.capitalize()}: F1={f1_score:.4f}")
    print(f"{'='*80}")

# Executa análise completa se avaliação foi bem-sucedida
if 'metrics' in locals() and metrics is not None:
    create_comprehensive_visualizations(history, confusion_matrix_result, metrics, detailed_report)
    print("Análise completa finalizada!")
    print(f"Arquivos salvos:")
    print(f"  • Métricas: metrics/performance_metrics.csv")
    print(f"  • Visualizações: plots/comprehensive_analysis_{experiment_id}.png")
    if model_saved:
        print(f"  • Modelo: models/weights_resnet50_{experiment_id}.h5")
else:
    print("Erro: Avaliação não foi executada corretamente")